# DATASET TRANSFORMATION


### Mind2Web data

    "annotation_id" (str): unique id for each task
    "website" (str): website name
    "domain" (str): website domain
    "subdomain" (str): website subdomain
    "confirmed_task" (str): task description
    "action_reprs" (list[str]): human readable string representation of the - action sequence
    "actions" (list[dict]): list of actions (steps) to complete the task
        "action_uid" (str): unique id for each action (step)
        "raw_html" (str): raw html of the page before the action is performed
        "cleaned_html" (str): cleaned html of the page before the action is performed
        "operation" (dict): operation to perform
            "op" (str): operation type, one of CLICK, TYPE, SELECT
            "original_op" (str): original operation type, contain additional HOVER and ENTER that are mapped to CLICK, not used
            "value" (str): optional value for the operation, e.g., text to type, option to select
        "pos_candidates" (list[dict]): ground truth elements. Here we only include positive elements that exist in "cleaned_html" after our preprocessing, so "pos_candidates" might be empty. The original labeled element can always be found in the "raw_html".
            "tag" (str): tag of the element
            "is_original_target" (bool): whether the element is the original target labeled by the annotator
            "is_top_level_target" (bool): whether the element is a top level target find by our algorithm. please see the paper for more details.
            "backend_node_id" (str): unique id for the element
            "attributes" (str): serialized attributes of the element, use json.loads to convert back to dict
        "neg_candidates" (list[dict]): other candidate elements in the page after preprocessing, has similar structure as "pos_candidates"


## LOAD TRANSFORM AND SAVE DATSET

#### LOADING

In [1]:
# Load the dataset from huggingface
from datasets import load_dataset

ds = load_dataset("osunlp/Mind2Web")
ds_train = ds['train']

/home/vicentamen/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### PARSING DATASET

In [30]:
# From the HTML of each task extract the nodes that are in the positive candidates and negative candidates and have text
from dataclasses import dataclass
from bs4 import BeautifulSoup, Tag

import re
# Class for storing all the node's data
@dataclass
class Node:
    def __init__(self, action_id,  node_id, text, pos_candidate, pos_candidate_k1 = False, pos_candidate_k3 = False, pos_candidate_k5 = False):
        self.action_id = action_id
        self.node_id = node_id
        self.text = text
        self.pos_candidate = pos_candidate
        if(pos_candidate):
            self.pos_candidate_k1 = True
            self.pos_candidate_k3 = True
            self.pos_candidate_k5 = True
        else:
            self.pos_candidate_k1 = pos_candidate_k1
            self.pos_candidate_k3 = pos_candidate_k3
            self.pos_candidate_k5 = pos_candidate_k5
        
    def to_dict(self):
        return {
            'action_id': self.action_id,
            'node_id': self.node_id,
            'text': self.text,
            'pos_candidate': self.pos_candidate,
            'pos_candidate_k1': self.pos_candidate_k1,
            'pos_candidate_k3': self.pos_candidate_k3,
            'pos_candidate_k5': self.pos_candidate_k5
        }
        
@dataclass
class Task:
    def __init__(self, task_id, prompt, nodes):
        self.task_id = task_id
        self.prompt = prompt
        self.nodes = nodes
        
    def to_dict(self):
        return {
            'task_id': self.task_id,
            'nodes': [node.to_dict() for node in self.nodes]
        }

# Extract the nodes from the HTML       
def extract_nodes(action_id, html, positive_candidates, negative_candidates):
    soup = BeautifulSoup(html, 'html.parser')
    nodes = []
    
    positive_candidates_id = {candidate['backend_node_id'] for candidate in positive_candidates}
    negative_candidates_id = {candidate['backend_node_id'] for candidate in negative_candidates}
    
    elements = soup.find_all()
    for element in elements:
        if element.has_attr('backend_node_id'):
            # Get the node id and the text
            #NOTE: THERE ARE SOME NODES THAT HAVE NOT TEXT OR ARIA_LABEL
            # WE COULD CLEAN UP  THE TEXT AND REMOVE STOPWORDS
            node_id = element.get('backend_node_id')
            node_text = re.sub(r'\s+', ' ', element.text).strip()
            if node_text == '' or node_text == 'NaN':
                node_text = element.get('aria_label')
                
            if node_id in positive_candidates_id:
                nodes.append(Node(action_id, node_id, node_text, True))
            elif node_id in negative_candidates_id:
                neg_candidate = Node(action_id,node_id, node_text, False)
                neg_candidate.pos_candidate_k1 = pos_to_the_kth(element, positive_candidates_id, 1)
                neg_candidate.pos_candidate_k3 = pos_to_the_kth(element, positive_candidates_id, 3)
                neg_candidate.pos_candidate_k5 = pos_to_the_kth(element, positive_candidates_id, 5)
                nodes.append(neg_candidate)
                
                
    return nodes

def pos_to_the_kth(element, pos_candidates_id, k=0):
    """This function checks if the element is a positive candidate up to the k-th
    parent or child of the element"""
    node_id = element.get('backend_node_id')
    
    # When k is 0 we check the element itself
    if k == 0:
        return node_id in pos_candidates_id

    # If the element is positive, we return True
    if node_id in pos_candidates_id:
        return True

    # Check the parent if we are still within depth k
    if element.parent is not None and isinstance(element.parent, Tag):
        if pos_to_the_kth(element.parent, pos_candidates_id, k - 1):
            return True

    # Check the children if we are still within depth k
    for child in element.contents:
        if isinstance(child, Tag):  # Ignore NavigableString objects
            if pos_to_the_kth(child, pos_candidates_id, k - 1):
                return True

    return False



# Extrac the data from a task
def extract_data_from_task(task):
    # Get the task data
    task_id = task['annotation_id']
    prompt = task['confirmed_task']
    
    nodes = []
    # Extract data from the task actions
    for action in task['actions']:
        pos_candidates = action['pos_candidates']
        neg_candidates = action['neg_candidates']
        html = action['cleaned_html']
        action_id = action['action_uid']
        
        nodes += extract_nodes(action_id, html, pos_candidates, neg_candidates)
    
    # We could make the prompt deconstructio before storing it or after
    return Task(task_id, prompt, nodes)
    
# Extract the data from the all the dataset and store it as a csv file
def extract_data(dataset):
    # Progress variables
    progress = 0
    total = len(dataset)
    # Init variables
    tasks = []
    # print progress bar
    print(f'Progress: {progress}/{total}', end='\r')
    for task in dataset:
        tasks.append(extract_data_from_task(task))
        progress += 1
        print(f'Progress: {progress}/{total}', end='\r')
        
    
    print('Data extracted from the dataset')
    return tasks


def store_data(tasks, nodes_path, prompts_path):
    import pandas as pd
    
    task_prompts_df = pd.DataFrame([{'task_id': task.task_id, 'prompt': task.prompt} for task in tasks])
    nodes_df = pd.DataFrame([{'task_id': task.task_id, 
                              'action_id': node.action_id,
                              'node_id': node.node_id, 'text': node.text, 
                              'pos_candidate': node.pos_candidate,
                              'pos_candidate_k1': node.pos_candidate_k1,
                              'pos_candidate_k3': node.pos_candidate_k3,
                              'pos_candidate_k5': node.pos_candidate_k5} for task in tasks for node in task.nodes])
    
    task_prompts_df.to_csv('dataset/task_prompts.csv', index=False)
    nodes_df.to_csv('dataset/nodes.csv', index=False)
    
    print('Data stored at dataset/task_prompts.csv and dataset/nodes.csv')

In [31]:
# Test data extraction from dataset
subset = ds_train.select(range(30))
tasks = extract_data(subset)
store_data(tasks, 'dataset/nodes.csv', 'dataset/task_prompts.csv')

Data extracted from the dataset
Data stored at dataset/task_prompts.csv and dataset/nodes.csv


#### EXTRACT DATASET DATA

In [25]:
# Default data
tasks = extract_data(ds_train)
store_data(tasks, 'dataset/nodes.csv', 'dataset/task_prompts.csv')

KeyboardInterrupt: 

## INDEXING

#### LOAD DATA

In [26]:
import pandas as pd
def load_data():
    task_prompts_df = pd.read_csv('dataset/task_prompts.csv')
    nodes_df = pd.read_csv('dataset/nodes.csv')
    
    return task_prompts_df, nodes_df

def generate_embeddings(task_prompts_df, nodes_df, model):
    task_prompts = task_prompts_df['prompt'].tolist()
    
    # Combine the 'tag' and 'text' columns for each node
    nodes_texts = nodes_df.apply(lambda row: f"tag: {row['tag']}, text: {row['text']}", axis=1).tolist()
    
    # Convert all elements to strings
    task_prompts = [str(prompt) for prompt in task_prompts]
    nodes_texts = [str(text) for text in nodes_texts]
    
    task_prompts_embeddings = model.encode(task_prompts)
    nodes_texts_embeddings = model.encode(nodes_texts)
    
    return task_prompts_embeddings, nodes_texts_embeddings

In [27]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

task_prompts_df, nodes_df = load_data()

2024-10-15 22:56:16.350248: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 22:56:16.457492: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-15 22:56:17.058537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 22:56:18.946304: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/vicentamen/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning

In [28]:
print(len(task_prompts_df), len(nodes_df))

30 161140


#### GENERATE EMBEDINGS

In [38]:
# Get only one task for testing
test_task = task_prompts_df.head(1)
test_nodes = nodes_df[nodes_df['task_id'] == test_task['task_id'].values[0]]

# Generate embedings for the test task
task_prompts_embeddings, nodes_texts_embeddings = generate_embeddings(test_task, test_nodes, model)

# Store embedings for testing evaluation
import numpy as np
np.save('dataset/task_prompts_embeddings.npy', task_prompts_embeddings)
np.save('dataset/nodes_texts_embeddings.npy', nodes_texts_embeddings)

print(task_prompts_embeddings.shape, nodes_texts_embeddings.shape)

(1, 768) (2076, 768)


In [ ]:
# Genereta emebeddings for the task prompts and nodes texts
task_prompts_embeddings, nodes_texts_embeddings = generate_embeddings(task_prompts_df, nodes_df, model)

# Store the embeddings
import numpy as np

np.save('dataset/task_prompts_embeddings.npy', task_prompts_embeddings)
np.save('dataset/nodes_texts_embeddings.npy', nodes_texts_embeddings)

print('Embeddings stored at dataset/task_prompts_embeddings.npy and dataset/nodes_texts_embeddings.npy')

## RETRIEVING

#### EMBEDINGS LOADING